In [9]:
import pandas as pd
import requests

In [189]:
#!pip install requests

In [491]:
#url = "https://api.openweathermap.org/data/3.0/onecall?lat=-37.840935&lon=144.946457&exclude=minutely&appid=c92389d6904463e3cb24208905434fd9"
url = "https://api.openweathermap.org/data/2.5/weather?lat=-37.840935&lon=144.946457&appid=c92389d6904463e3cb24208905434fd9"
response = requests.get(url)

In [571]:
json = response.json()

In [572]:
weather = json['weather'][0]['main']

In [635]:
weather = 'Rain'

In [636]:
data = pd.read_csv('outdoor.csv')

In [637]:
data = data.drop(['Unnamed: 0','Rating'], axis = 1)

In [638]:
data.head()

,Title,Theme,Sub Theme,Latitude,Longitude,Green Space,id,Walking,Cardio,Sightseeing
0,Lynch''''s Bridge Mosaic Mural,Mural,Sculpture,144.923939,-37.790205,False,1001,False,False,True
1,Coles Fountain,Fountain,Memorials,144.973484,-37.809771,False,1002,False,False,True
2,Beyond the Ocean of Existence,Sculpture,Sculpture,144.967062,-37.816424,False,1003,False,False,True
3,Vault,Sculpture,Sculpture,144.967733,-37.826071,False,1004,False,False,True
4,Sir Thomas Blamey Memorial,Memorial,Sculpture,144.972819,-37.826301,False,1005,False,False,True


In [639]:
id = [1011, 1074, 1273, 1180, 2002]
Rating = [2,2, 1, 1, 2]
user = pd.DataFrame()
user["id"] = id
user["Rating"] = Rating

In [640]:
pref = ['Walking', 'Cardio', 'Cycling', 'Dog']
if 'Cycling' in pref:
    if weather == 'Rain':
        n = 2
    else:
        n = 3
elif 'Dog' in pref:
    if weather == 'Rain':
        n = 3
    else: 
        n = 2
else:
    if weather == 'Rain':
        n = 2
    else:
        n = 3

In [641]:
user['Rating'].dtype

dtype('int64')

In [642]:
temp = data
merged = pd.merge(temp, user, on = 'id', how = 'left')
merged['Rating'] = merged['Rating'].fillna(0)
merged = merged.astype({'Rating': 'int64'})

In [643]:
temp1 = pd.DataFrame(columns = ["Title", "Theme","Sub Theme","Latitude","Longitude","Green Space","id","Walking","Cardio","Sightseeing","Rating"])
temp2 = pd.DataFrame(columns = ["Title", "Theme","Sub Theme","Latitude","Longitude","Green Space","id","Walking","Cardio","Sightseeing","Rating"])
temp3 = pd.DataFrame(columns = ["Title", "Theme","Sub Theme","Latitude","Longitude","Green Space","id","Walking","Cardio","Sightseeing","Rating"])
if 'Walking' in pref:
    temp1 = merged[merged.Walking != False]
if 'Cardio' in pref:
    temp2 = merged[merged.Cardio != False]
if 'Sightseeing' in pref:
    temp3 = merged[merged.Sightseeing != False]

000
001
011
111
010
100
110
101

In [644]:
concat = pd.concat([temp1, temp2, temp3])

In [645]:
if concat.empty:
    concat = temp

In [646]:
# from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
if len(concat[(concat['Rating'] != 0)]) < 50:
    df_elements = concat.sample(n)
else:
#     len(concat[(concat['Rating'] != 0)]) > 50
#     train, test = train_test_split(concat, test_size = 0.2)
    concat["Theme"] = concat["Theme"].astype('category')
    d1 = dict(enumerate(concat['Theme'].cat.categories))
    concat["Sub Theme"] = concat["Sub Theme"].astype('category')
    d2 = dict(enumerate(concat['Sub Theme'].cat.categories))
    concat["Theme"] = concat["Theme"].cat.codes
    concat["Sub Theme"] = concat["Sub Theme"].cat.codes

    train = concat.loc[concat['Rating'] != 0]
    test = concat.loc[concat['Rating'] == 0]
    x_train = train.loc[:, ~train.columns.isin(['Rating', 'Title'])]
    y_train = train['Rating']
    x_test = test.loc[:, ~test.columns.isin(['Rating', 'Title'])]
    y_test = test['Rating']
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    y_pred = gnb.predict(x_test)
    train['Rating'] = y_train
    test['Rating'] = y_pred

    pred = pd.concat([train, test])
    pred = pred.loc[pred['Rating'] == 2]
    pred['Theme'] = pred['Theme'].map(d1)
    pred['Sub Theme'] = pred['Sub Theme'].map(d2)
    df_elements = pred.sample(n)
df_elements = df_elements.drop(['Walking', 'Cardio', 'Sightseeing', 'Green Space'], axis = 1)

In [647]:
df_elements

,Title,Theme,Sub Theme,Latitude,Longitude,id,Rating
243,Darling Square,Leisure/Recreation,Informal Outdoor Facility (Park/Garden/Reserve),144.989063,-37.812992,1244,0
198,Carlton Football Club,Leisure/Recreation,Major Sports & Recreation Facility,144.961968,-37.784086,1199,0


In [648]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

        # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r   

In [649]:
if 'Cycling' in pref:
    bike = pd.read_csv('bicycle.csv')
    bike.drop(['Unnamed: 0'], axis = 1)
#     bike['Rating'] = 0
    
    temp = bike
    bikemerged = pd.merge(temp, user, on = 'id', how = 'left')
    bikemerged['Rating'] = bikemerged['Rating'].fillna(0)
    bike_high = bikemerged.loc[bikemerged['Rating'] == 2]
    selection = bike_high.sample(1)
    if selection.empty:
        record = bike.sample(1)
    else:
        lon, lat = selection['Longitude'], selection['Latitude'] 
        bikemerged['haversine_calc'] = [haversine(lon,lat, bikemerged['Longitude'][i], bikemerged['Latitude'][i])for i in range(len(bike))]
        bikemerged = bikemerged[bikemerged["haversine_calc"]>0]
        bikemerged = bikemerged.drop(['Unnamed: 0'], axis = 1)
        record = bikemerged.loc[bikemerged.haversine_calc == bikemerged.haversine_calc.min()]
        record = record.drop(['haversine_calc'], axis = 1)

In [650]:
output = pd.concat([df_elements, record])

In [651]:
indoor = pd.DataFrame()
Title = ['Yoga', 'Skipping a rope', 'Body Weight Training', 'Stretching', 'Taking the stairs', 'Dancing', 'Hula Hoop', 'Pilates']
Theme = ['Low Intensity', 'High Intensity', 'Low Intensity', 'Low Intensity', 'High Intensity', 'High Intensity', 'High Intensity', 'Low Intensity']
SubTheme = ['Meditation', 'Endurance Training', 'Muscle Building', 'Muscle Building', 'Endurance Training', 'Endurance Training', 'Endurance Training', 'Meditation']
id = [3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008]
indoor['Title'] = Title
indoor['Theme'] = Theme
indoor['Sub Theme'] = SubTheme
indoor['id'] = id

In [652]:
indoor

,Title,Theme,Sub Theme,id
0,Yoga,Low Intensity,Meditation,3001
1,Skipping a rope,High Intensity,Endurance Training,3002
2,Body Weight Training,Low Intensity,Muscle Building,3003
3,Stretching,Low Intensity,Muscle Building,3004
4,Taking the stairs,High Intensity,Endurance Training,3005
5,Dancing,High Intensity,Endurance Training,3006
6,Hula Hoop,High Intensity,Endurance Training,3007
7,Pilates,Low Intensity,Meditation,3008


In [653]:
if 'Cardio' in pref:
    indoor_act = indoor.loc[indoor['Theme'] == 'High Intensity']
    indoor_act = indoor_act.sample(3)
else:
    indoor_act = indoor.loc[indoor['Theme'] == 'Low Intensity']
    indoor_act = indoor_act.sample(3)
    
if weather == 'Rain':
    indoor_act = indoor_act.sample(3)
else:
    indoor_act = indoor_act.sample(2)

In [654]:
output_final = pd.concat([output, indoor_act])

In [1]:
output_final

NameError: name 'output_final' is not defined

In [200]:
### TESTING DATASET
from random import randint
concat["Rating"] = concat.apply(lambda x: randint(0,2), axis=1)
concat = concat.drop(['Title'], axis = 1)
concat["Theme"] = concat["Theme"].astype('category')
concat["Sub Theme"] = concat["Sub Theme"].astype('category')
concat["Theme"] = concat["Theme"].cat.codes
concat["Sub Theme"] = concat["Sub Theme"].cat.codes
train = concat.loc[concat['Rating'] != 0]
test = concat.loc[concat['Rating'] == 0]
x_train = train.loc[:, train.columns != 'Rating']
y_train = train['Rating']
x_test = test.loc[:, test.columns != 'Rating']
y_test = test['Rating']

In [201]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)

GaussianNB()

In [202]:
y_pred = gnb.predict(x_test)

In [203]:
y_pred

array([1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2],
      dtype=int64)

In [351]:
from sklearn.model_selection import train_test_split
concat["Rating"] = concat.apply(lambda x: randint(0,2), axis=1)

concat["Theme"] = concat["Theme"].astype('category')
d1 = dict(enumerate(concat['Theme'].cat.categories))
concat["Sub Theme"] = concat["Sub Theme"].astype('category')
d2 = dict(enumerate(concat['Sub Theme'].cat.categories))

concat["Theme"] = concat["Theme"].cat.codes
concat["Sub Theme"] = concat["Sub Theme"].cat.codes
train = concat.loc[concat['Rating'] != 0]
train, test = train_test_split(train, test_size = 0.3, random_state = 1)
x_train = train.loc[:, ~train.columns.isin(['Rating', 'Title'])]
y_train = train['Rating']
x_test = test.loc[:, ~test.columns.isin(['Rating', 'Title'])]
y_test = test['Rating']

In [352]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)

GaussianNB()

In [353]:
y_pred = gnb.predict(x_test)

In [354]:
from sklearn import metrics
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Gaussian Naive Bayes model accuracy(in %): 30.0


In [355]:
train['Rating'] = y_train
test['Rating'] = y_pred
pred = pd.concat([train, test])
pred = pred.loc[pred['Rating'] == 2]
pred['Theme'] = pred['Theme'].map(d1)
pred['Sub Theme'] = pred['Sub Theme'].map(d2)
df_elements = pred.sample(n)

In [356]:
df_elements

,Title,Theme,Sub Theme,Latitude,Longitude,Green Space,id,Walking,Cardio,Sightseeing,Rating
230,Artplay,Leisure/Recreation,Indoor Recreation Facility,144.971543,-37.818420,True,1231,False,True,False,2
221,Newmarket Reserve,Leisure/Recreation,Informal Outdoor Facility (Park/Garden/Reserve),144.922972,-37.787847,True,1222,True,False,False,2
194,Carlton Gardens South,Leisure/Recreation,Informal Outdoor Facility (Park/Garden/Reserve),144.971266,-37.806068,True,1195,True,False,False,2
263,City Baths,Leisure/Recreation,Major Sports & Recreation Facility,144.963190,-37.807159,True,1264,False,True,False,2


In [357]:
d1

{0: 'Leisure/Recreation'}